# Preprocess MRI
## Packages

In [ ]:
import pydicom as dicom
import numpy as np
import matplotlib.pyplot as plt
import os
import nibabel as nib
from ipywidgets import interact
from matplotlib.colors import ListedColormap

### Helper Functions

In [8]:
def get_mri_slices_from_path(path):
    path_list = os.listdir(path)
    slices = []
    for dcm_path in path_list:
        try:
            slice = dicom.dcmread(path + "/" + dcm_path)  
            slices.append(slice)
        except:
            pass
    return slices

def get_slice_descriptions(slices):
    names = [ds.SeriesDescription for ds in slices]
    print(np.unique(names,return_counts=True))

def slices_sort_and_selectet(slices,slice_type):
    slices.sort(key=lambda x: int(x.SliceLocation))
    type = [slic.SeriesDescription == slice_type for slic in slices]
    slice_one_type = np.array(slices)[type]
    return slice_one_type

def visualize_slices(slices):
    cmap = plt.cm.winter
    # Get the colormap colors
    my_cmap = cmap(np.arange(cmap.N))
    my_cmap[:,-1] = np.linspace(0, 1, cmap.N)
    my_cmap = ListedColormap(my_cmap)

    # Visualize Images as a slider
    def show_slice(i):
        plt.imshow(slices[i].pixel_array, cmap='gray')
        plt.show()

    interact(show_slice, i=(0, len(slices)-1))
def pad_array(original_array,axis,padd_size):

    # Calculate padding
    total_padding = padd_size - 15 
    padding_before = total_padding // 2 
    padding_after = total_padding - padding_before
    if axis == 0:
        padding = ((padding_before, padding_after),(0, 0), (0, 0))
    elif axis == 1:
        padding = ((0, 0),(padding_before, padding_after), (0, 0))
    elif axis == 2:
        padding = ((0, 0),(0, 0), (padding_before, padding_after))
    # Pad the array
    padded_array = np.pad(
        original_array,
        padding,
        mode='constant',
        constant_values=0
    )
    return padded_array

In [17]:
from azureml.core import Workspace, Dataset, Datastore
subscription_id = '35068096-667c-4620-8bb2-6b050765fd4e'
resource_group = 'ComputeFHNW'
workspace_name = 'FHNW'
  
workspace = Workspace(subscription_id, resource_group, workspace_name)
  
datastore = Datastore.get(workspace, "mrialldata")
image_folder = "7013955/"
folder_path = 'output.deid/images/7013955/*'
dataset = Dataset.File.from_files(path=(datastore, folder_path))
dataset.download(target_path='../pro5d-classification-prolactinoma/raw_data/downloaded_mri_files/' + image_folder, overwrite=True)

Engine process terminated with returncode=-2


KeyboardInterrupt: 

In [18]:
from azureml.core import Workspace, Dataset, Datastore

subscription_id = '35068096-667c-4620-8bb2-6b050765fd4e'
resource_group = 'ComputeFHNW'
workspace_name = 'FHNW'

workspace = Workspace(subscription_id, resource_group, workspace_name)

datastore = Datastore.get(workspace, "mr_testdata")
dataset = Dataset.File.from_files(path=(datastore, '8797386'))
dataset.download(target_path='../pro5d-classification-prolactinoma/raw_data/downloaded_mri_files/8797386/', overwrite=True)

In [23]:
slices = get_mri_slices_from_path("../raw_data/MRI")

In [ ]:
get_slice_descriptions(slices)

(array(['ep2d_diff_tra_ADC', 'ep2d_diff_tra_TRACEW', 't1_mprage_we_sag',
       't1_mprage_we_sag_MPR_cor', 't1_mprage_we_sag_MPR_tra',
       't1_tse_cor', 't1_tse_fs_cor', 't1_tse_sag', 't2_tse_fs_cor',
       't2_tse_fs_sag'], dtype='<U24'), array([ 30,  60, 208, 200, 210,  15,  15,  15,  15,  15]))


In [ ]:
slice_t2_cor = slices_sort_and_selectet(slices,"t2_tse_fs_cor")
visualize_slices(slice_t2_cor) # 7 - 9 Relevant evtl bei 125 - 250 y-achse 100 - 300 x- achse

interactive(children=(IntSlider(value=7, description='i', max=14), Output()), _dom_classes=('widget-interact',…

In [ ]:
slice_t1_cor = slices_sort_and_selectet(slices,"t1_tse_fs_cor")
visualize_slices(slice_t1_cor) # (6) 7,8 (9)

interactive(children=(IntSlider(value=7, description='i', max=14), Output()), _dom_classes=('widget-interact',…

In [ ]:
t1_tse_cor= slices_sort_and_selectet(slices,"t1_tse_cor")
visualize_slices(t1_tse_cor)

interactive(children=(IntSlider(value=7, description='i', max=14), Output()), _dom_classes=('widget-interact',…

In [ ]:
slice_t2_sag= slices_sort_and_selectet(slices,"t2_tse_fs_sag")
visualize_slices(slice_t2_sag) # 4-9 y-achse 125 - 250 x achse 100 -300

interactive(children=(IntSlider(value=7, description='i', max=14), Output()), _dom_classes=('widget-interact',…

In [131]:
# Read the DICOM files and stack them into a single 3D numpy array
array_list = np.array([dcm.pixel_array for dcm in slices_sort_and_selectet(slices,"t1_mprage_we_sag_MPR_cor")]).transpose((0,1,2))
#array_list = pad_array(array_list,axis= 1,padd_size=150)
array_list= np.rot90(array_list,k=45, axes=(1,2))
print(array_list.shape)
volume_3d = np.stack(array_list, axis=-1)

# Create a NIfTI image (you might need to provide the correct affine transformation)
affine = np.eye(4)  # This is a placeholder; you may need the actual affine from DICOM headers
nifti_img = nib.Nifti1Image(volume_3d, affine)

# Save the NIfTI image to a file
nib.save(nifti_img, '../raw_data/nii_files/test.nii')

(200, 267, 267)


In [132]:
def show_slice(i):
    plt.imshow(array_list[:,:,i], cmap='gray')
    plt.show()

interact(show_slice, i=(0, array_list.shape[2]-1))

interactive(children=(IntSlider(value=133, description='i', max=266), Output()), _dom_classes=('widget-interac…

<function __main__.show_slice(i)>